<a href="https://colab.research.google.com/github/xhr0804/BotBuilder-Samples/blob/master/tensorflow2/NER_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!free -g

              total        used        free      shared  buff/cache   available
Mem:             25           0          22           0           2          24
Swap:             0           0           0


In [2]:
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.train
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testa

--2020-12-05 02:40:59--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3281528 (3.1M) [text/plain]
Saving to: ‘eng.train.2’

eng.train.2         100%[===================>]   3.13M  --.-KB/s    in 0.1s    

2020-12-05 02:40:59 (28.4 MB/s) - ‘eng.train.2’ saved [3281528/3281528]

--2020-12-05 02:40:59--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 827012 (808K) [text/plain]
Savin

In [3]:
!ls -ls

total 12052
 808 -rw-r--r-- 1 root root  827012 Dec  5 02:18 eng.testa
 808 -rw-r--r-- 1 root root  827012 Dec  5 02:33 eng.testa.1
 808 -rw-r--r-- 1 root root  827012 Dec  5 02:41 eng.testa.2
3208 -rw-r--r-- 1 root root 3281528 Dec  5 02:18 eng.train
3208 -rw-r--r-- 1 root root 3281528 Dec  5 02:33 eng.train.1
3208 -rw-r--r-- 1 root root 3281528 Dec  5 02:40 eng.train.2
   4 drwxr-xr-x 1 root root    4096 Dec  2 22:04 sample_data


In [4]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import time
import matplotlib.pyplot as plt

print(tf.__version__)
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.0


In [5]:
def countWord(file):
  word_count = {}
  tag_count = {}
  for line in open(file):
    if '-DOCSTART-' in line or not len(line.strip('\n')):
      continue
    splitted = line.split()
    if splitted[0] in word_count:
      word_count[splitted[0]] += 1
    else:
      word_count[splitted[0]] = 1
    if splitted[-1] in tag_count:
      tag_count[splitted[-1]] += 1
    else:
      tag_count[splitted[-1]] = 1
  return word_count, tag_count

In [6]:
word_count, tag_count = countWord('eng.train')
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
word2idx = {kv[0]: index+2 for index, kv in enumerate(sorted_word_count)}
word2idx["<PAD>"] = 0
word2idx["<UNK>"] = 1
idx2word = {idx:word for word, idx in word2idx.items()}
sorted_tag_count = sorted(tag_count.items(), key=lambda x: x[1], reverse=True)
tag2idx = {kv[0]: index for index, kv in enumerate(sorted_tag_count)}
idx2tag = {idx:tag for tag, idx in tag2idx.items()}

In [7]:
def parse(file):
  left, right = [], []
  for line in open(file):
    line = line.strip()
    if '-DOCSTART-' in line:
      this_left, this_right = [], []
    elif not len(line):
      if len(this_left) > 0:
        left.append(this_left)
        right.append(this_right)
      this_left, this_right = [], []
    else:
      splitted = line.split()
      this_left.append(splitted[0])
      this_right.append(splitted[-1])
  return left, right

In [8]:
left_train, right_train = parse('eng.train')
left_test, right_test = parse('eng.testa')

In [9]:
print(left_train[:5])
print(right_train[:5])
print(len(left_train))

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn'], ['BRUSSELS', '1996-08-22'], ['The', 'European', 'Commission', 'said', 'on', 'Thursday', 'it', 'disagreed', 'with', 'German', 'advice', 'to', 'consumers', 'to', 'shun', 'British', 'lamb', 'until', 'scientists', 'determine', 'whether', 'mad', 'cow', 'disease', 'can', 'be', 'transmitted', 'to', 'sheep', '.'], ['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']]
[['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O'], ['I-PER', 'I-PER'], ['I-LOC', 'O'], ['O', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['I-LOC', 'O

In [10]:
def convertWord2Idx(words, map, threshold):
  ret = []
  for word in words:
    if word in map and map[word] < threshold:
      ret.append(map[word])
    else:
      ret.append(map["<UNK>"])
  return np.array(ret, dtype=np.int32)

In [11]:
x_train = np.array([convertWord2Idx(words, word2idx, 10000) for words in left_train])
y_train = np.array([convertWord2Idx(words, tag2idx, 10000) for words in right_train])
x_test = np.array([convertWord2Idx(words, word2idx, 10000) for words in left_test])
y_test = np.array([convertWord2Idx(words, tag2idx, 10000) for words in right_test])

In [12]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train,
                                                     padding='post',
                                                     value = word2idx["<PAD>"],
                                                     maxlen = 64)

In [13]:
y_train = keras.preprocessing.sequence.pad_sequences(y_train,
                                                     padding='post',
                                                     value = 0,
                                                     maxlen = 64)

In [14]:
# print(x_train[:3])
# print(y_train[:3])

In [15]:
print(type(x_train))
print(type(x_train[0]))
print(type(x_train[0][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.int32'>


In [16]:
print(type(y_train))
print(type(y_train[0]))
print(type(y_train[0][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.int32'>


In [17]:
# print(x_train[:5])
# print(y_train[:5])

In [18]:
model = keras.Sequential([
  keras.layers.Embedding(10000, 16, mask_zero=True), #padding/mask https://www.tensorflow.org/guide/keras/masking_and_padding
  keras.layers.LSTM(16, return_sequences=True),
  keras.layers.Dense(8)
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
lstm (LSTM)                  (None, None, 16)          2112      
_________________________________________________________________
dense (Dense)                (None, None, 8)           136       
Total params: 162,248
Trainable params: 162,248
Non-trainable params: 0
_________________________________________________________________


In [20]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [21]:
model.compile(optimizer='adam', 
              # if use CategoricalCrossentropy(), should be one hot.
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [22]:
for i in y_train:
  for y in i:
    if type(y) != type(x_train[0][0]):
      print(y)
      break;

In [23]:
x_train[:2]

array([[ 958,    1,  234,  763,    7, 4148,  210, 6183,    2,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 733, 2069,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [24]:
y_train[:2]

array([[2, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)

In [25]:
model.fit(
    x = x_train[:12000], # batchsize * sentence_len(word index)
    y = y_train[:12000], # batchsize * sentence_len(tag index)
    batch_size = 32,
    epochs = 40,
    validation_data = (x_train[12000:], y_train[12000:]),
    callbacks = [earlystopping]
)

Epoch 1/40
375/375 [==============================] - 13s 34ms/step - loss: 0.1883 - accuracy: 0.8224 - val_loss: 0.1239 - val_accuracy: 0.8620
Epoch 2/40
375/375 [==============================] - 11s 30ms/step - loss: 0.1140 - accuracy: 0.8427 - val_loss: 0.0989 - val_accuracy: 0.8733
Epoch 3/40
375/375 [==============================] - 11s 30ms/step - loss: 0.0709 - accuracy: 0.8797 - val_loss: 0.1001 - val_accuracy: 0.8972
Epoch 4/40
375/375 [==============================] - 11s 30ms/step - loss: 0.0758 - accuracy: 0.8781 - val_loss: 0.0963 - val_accuracy: 0.8606
Epoch 5/40
375/375 [==============================] - 11s 30ms/step - loss: 0.0721 - accuracy: 0.8523 - val_loss: 0.0915 - val_accuracy: 0.8732
Epoch 6/40
375/375 [==============================] - 11s 30ms/step - loss: 0.0627 - accuracy: 0.8699 - val_loss: 0.0895 - val_accuracy: 0.8847
Epoch 7/40
375/375 [==============================] - 11s 30ms/step - loss: 0.0555 - accuracy: 0.8883 - val_loss: 0.0883 - val_accuracy:

In [36]:
x_pred = model.predict(x_train[:10, :10])

In [27]:
x_pred[0]

array([[-0.09160127, -0.09651791,  0.1363525 , -0.08148149, -0.0534771 ,
        -0.04810348, -0.15439141, -0.15556622],
       [ 0.10157729, -0.00080369,  0.10751305, -0.0809681 , -0.03987847,
        -0.07787485, -0.13765295, -0.13717373],
       [-0.04216807, -0.04479156, -0.00328986, -0.16247433,  0.1022789 ,
        -0.12491561, -0.20857745, -0.20520884],
       [ 0.18904963, -0.07787696, -0.2401216 , -0.25603452, -0.1519022 ,
        -0.13334757, -0.2588584 , -0.20988387],
       [ 0.24817798, -0.11924468, -0.28075442, -0.21741584, -0.15434244,
        -0.13063264, -0.29326335, -0.2093984 ],
       [ 0.23242828, -0.11067462, -0.26406065, -0.20214431, -0.08678062,
        -0.13031885, -0.28472108, -0.21106806],
       [-0.00459761, -0.05415946, -0.13124481,  0.02191197,  0.19112122,
        -0.14641052, -0.30423757, -0.19865073],
       [ 0.24196947, -0.17153585, -0.24138421, -0.15761752, -0.08175099,
        -0.13587892, -0.29605454, -0.22176164],
       [ 0.4179797 , -0.29753563

In [28]:
pred = tf.argmax(x_pred, axis=2)

In [29]:
pred

<tf.Tensor: shape=(10, 10), dtype=int64, numpy=
array([[2, 2, 4, 0, 0, 0, 4, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 4, 2, 0, 0, 0, 0, 0, 0, 4],
       [3, 0, 0, 0, 0, 4, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 3, 0, 3],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 2]])>

In [30]:
y_train[:10, :10]

array([[2, 0, 4, 0, 0, 0, 4, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 2, 0, 0, 0, 0, 0, 0, 4],
       [3, 0, 0, 0, 0, 2, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
       [1, 0, 4, 0, 0, 0, 0, 3, 0, 3],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 2]], dtype=int32)

In [31]:
def decode(pred_tag_index, idx2tag, sentence):
  ret = [sentence[i] + "(" + idx2tag[pred_tag_index[i]] + ")" for i in range(len(sentence))]
  print(" ".join(ret))

In [32]:
decode(pred[0].numpy(), idx2tag, left_train[0])

EU(I-ORG) rejects(I-ORG) German(I-MISC) call(O) to(O) boycott(O) British(I-MISC) lamb(O) .(O)


In [33]:
pred[0][0].numpy()

2

In [61]:
x_test = keras.preprocessing.sequence.pad_sequences(x_test,
                                                     padding='post',
                                                     value = word2idx["<PAD>"],
                                                     maxlen = 64)

In [62]:
y_test = keras.preprocessing.sequence.pad_sequences(y_test,
                                                     padding='post',
                                                     value = 0,
                                                     maxlen = 64)

In [67]:
x_test.shape

(3250, 64)

In [50]:
x_train[0]

array([ 958,    1,  234,  763,    7, 4148,  210, 6183,    2,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [49]:
x_test[0]

array([ 451,   18,    1, 9043, 8348,  127, 7587, 1827,    1,    1,    2],
      dtype=int32)

In [68]:
x_pred = model.predict(x_test)

In [69]:
pred = tf.argmax(x_pred, axis=2)

In [70]:
pred.shape

TensorShape([3250, 64])

In [71]:
pred[:10, :10]

<tf.Tensor: shape=(10, 10), dtype=int64, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
       [3, 3, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
       [2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 3, 0, 2, 0, 1, 1, 0, 0]])>

In [ ]:
pred = pred.numpy()

In [42]:
y_test[:10,:10]

array([[0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 4, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
       [2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 3, 0, 2, 0, 1, 1, 0, 0]], dtype=int32)

In [43]:
from sklearn.metrics import classification_report

In [75]:
pred[64][62]

0

In [76]:
pred_label = []
for i in range(len(right_test)):
  pred_label += [idx2tag[pred[i][j]] for j in range(min(len(right_test[i]), 64))]

In [80]:
print(pred_label[:100])
print(len(pred_label))

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'I-ORG']
51175


In [81]:
real_label = []
for i in range(len(right_test)):
  real_label += [idx2tag[y_test[i][j]] for j in range(min(len(right_test[i]), 64))]

In [82]:
print(real_label[:100])
print(len(real_label))

['O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'I-MISC', 'I-MISC', 'O', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'I-LOC', 'O', 'I-ORG']
51175


In [83]:
print(classification_report(pred_label, real_label))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      B-MISC       0.00      0.00      0.00         0
       I-LOC       0.75      0.80      0.77      1947
      I-MISC       0.64      0.87      0.74       931
       I-ORG       0.62      0.83      0.71      1557
       I-PER       0.59      0.94      0.73      1947
           O       0.99      0.94      0.97     44793

    accuracy                           0.93     51175
   macro avg       0.60      0.73      0.65     51175
weighted avg       0.95      0.93      0.94     51175



In [46]:
np.array([[1,2,3], [3,4]])

array([list([1, 2, 3]), list([3, 4])], dtype=object)